## Trigger Joins

In [1]:
import json

def get_request_string(use_trigger_1=True, use_trigger_2=True):
    request = {
        "model_name": "whatever",
        "inputs": [
            {"name": "INPUT1", "contents": {"int64Contents": [1]}, "datatype": "INT64", "shape": [1]},
            {"name": "INPUT2", "contents": {"int64Contents": [1]}, "datatype": "INT64", "shape": [1]},
        ]
    }

    if use_trigger_1:
        request["inputs"].append({"name": "TRIGGER1", "contents": {"boolContents": [True]}, "datatype": "BOOL", "shape": [1]})
    
    if use_trigger_2:
        request["inputs"].append({"name": "TRIGGER2", "contents": {"boolContents": [True]}, "datatype": "BOOL", "shape": [1]})
    
    request_string = json.dumps(request)
    return request_string

Load models and pipelines

In [2]:
!seldon model load -f ./models/id1_node.yaml
!seldon model load -f ./models/id2_node.yaml
!seldon model load -f ./models/join_node.yaml

{}
{}
{}


In [3]:
!seldon model status join_node -w ModelAvailable
!seldon model status id1_node -w ModelAvailable
!seldon model status id2_node -w ModelAvailable

{}
{}
{}


In [4]:
!seldon pipeline load -f ./pipelines/triggers_join_inputs.yaml

{}


In [5]:
!seldon pipeline status triggers_join_inputs -w PipelineReady | jq .

{
  "pipelineName": "triggers_join_inputs",
  "versions": [
    {
      "pipeline": {
        "name": "triggers_join_inputs",
        "uid": "cbmbfial0p8os8jr7ds0",
        "version": 1,
        "steps": [
          {
            "name": "join_node",
            "inputs": [
              "triggers_join_inputs.inputs.INPUT1",
              "triggers_join_inputs.inputs.INPUT2"
            ],
            "triggers": [
              "triggers_join_inputs.inputs.TRIGGER1",
              "triggers_join_inputs.inputs.TRIGGER2"
            ],
            "triggersJoin": "ANY"
          }
        ],
        "output": {
          "steps": [
            "join_node.outputs"
          ]
        },
        "kubernetesMeta": {}
      },
      "state": {
        "pipelineVersion": 1,
        "status": "PipelineReady",
        "reason": "Created pipeline",
        "lastChangeTimestamp": "2022-08-05T06:25:14.011256855Z"
      }
    }
  ]
}


In [6]:
!cat ./pipelines/triggers_join_inputs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: triggers_join_inputs
spec:
  steps:
    - name: join_node
      inputs:
        - triggers_join_inputs.inputs.INPUT1
        - triggers_join_inputs.inputs.INPUT2
      triggers:
        - triggers_join_inputs.inputs.TRIGGER1
        - triggers_join_inputs.inputs.TRIGGER2
      triggersJoinType: any
  output:
    steps:
      - join_node


In [7]:
request_string = get_request_string(use_trigger_1=True, use_trigger_2=True)

!seldon pipeline infer triggers_join_inputs --inference-mode grpc '{request_string}'

{"outputs":[{"name":"OUTPUT1","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["2"]}}],"rawOutputContents":["AgAAAAAAAAA="]}


In [8]:
request_string = get_request_string(use_trigger_1=True, use_trigger_2=False)

!seldon pipeline infer triggers_join_inputs --inference-mode grpc '{request_string}'

{"outputs":[{"name":"OUTPUT1","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["2"]}}],"rawOutputContents":["AgAAAAAAAAA="]}


In [9]:
request_string = get_request_string(use_trigger_1=False, use_trigger_2=True)

!seldon pipeline infer triggers_join_inputs --inference-mode grpc '{request_string}'

{"outputs":[{"name":"OUTPUT1","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["2"]}}],"rawOutputContents":["AgAAAAAAAAA="]}


In [10]:
!seldon pipeline unload triggers_join_inputs

{}


In [11]:
!seldon pipeline load -f ./pipelines/triggers_join_internal.yaml

{}


In [12]:
!seldon pipeline status triggers_join_internal -w PipelineReady | jq .

{
  "pipelineName": "triggers_join_internal",
  "versions": [
    {
      "pipeline": {
        "name": "triggers_join_internal",
        "uid": "cbmbfm2l0p8os8jr7dsg",
        "version": 1,
        "steps": [
          {
            "name": "id1_node",
            "inputs": [
              "triggers_join_internal.inputs.TRIGGER1"
            ],
            "tensorMap": {
              "triggers_join_internal.inputs.TRIGGER1": "INPUT1"
            }
          },
          {
            "name": "id2_node",
            "inputs": [
              "triggers_join_internal.inputs.TRIGGER2"
            ],
            "tensorMap": {
              "triggers_join_internal.inputs.TRIGGER2": "INPUT1"
            }
          },
          {
            "name": "join_node",
            "inputs": [
              "triggers_join_internal.inputs.INPUT1",
              "triggers_join_internal.inputs.INPUT2"
            ],
            "triggers": [
              "id1_node.outputs.OUTPUT1",
              "id

In [13]:
!cat ./pipelines/triggers_join_internal.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: triggers_join_internal
spec:
  steps:
    - name: id1_node
      inputs:
        - triggers_join_internal.inputs.TRIGGER1
      tensorMap:
        triggers_join_internal.inputs.TRIGGER1: INPUT1
    - name: id2_node
      inputs:
        - triggers_join_internal.inputs.TRIGGER2
      tensorMap:
        triggers_join_internal.inputs.TRIGGER2: INPUT1
    - name: join_node
      inputs:
        - triggers_join_internal.inputs.INPUT1
        - triggers_join_internal.inputs.INPUT2
      triggers:
        - id1_node.outputs.OUTPUT1
        - id2_node.outputs.OUTPUT1
      triggersJoinType: any
  output:
    steps:
      - join_node


In [14]:
request_string = get_request_string(use_trigger_1=True, use_trigger_2=True)

!seldon pipeline infer triggers_join_internal --inference-mode grpc '{request_string}'

{"outputs":[{"name":"OUTPUT1","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["2"]}}],"rawOutputContents":["AgAAAAAAAAA="]}


In [15]:
request_string = get_request_string(use_trigger_1=True, use_trigger_2=False)

!seldon pipeline infer triggers_join_internal --inference-mode grpc '{request_string}'

{"outputs":[{"name":"OUTPUT1","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["2"]}}],"rawOutputContents":["AgAAAAAAAAA="]}


In [16]:
request_string = get_request_string(use_trigger_1=False, use_trigger_2=True)

!seldon pipeline infer triggers_join_internal --inference-mode grpc '{request_string}'

{"outputs":[{"name":"OUTPUT1","datatype":"INT64","shape":["1"],"contents":{"int64Contents":["2"]}}],"rawOutputContents":["AgAAAAAAAAA="]}


In [17]:
!seldon pipeline unload triggers_join_internal

{}


In [18]:
!seldon model unload id1_node
!seldon model unload id2_node
!seldon model unload join_node

{}
{}
{}
